In [ ]:
#Programmatically upload to S3 bucket
import boto3
import os

aws_access_key = os.environ['ACCESS_KEY']
aws_secret_key = os.environ['SECRET_KEY']

bucket_name = 'pep2-group1-etl-endpoint'

file_key = 'Outpatient_Respiratory_Illness_Activity_Map'

local_file_path = 'Healthcare-Data-Analysis/Outpatient_Respiratory_Illness_Activity_Map_20240205.csv'

s3 = boto3.client('s3', aws_access_key_id=aws_access_key, aws_secret_access_key=aws_secret_key)

with open(local_file_path, 'rb') as data:
    s3.put_object(Body=data, Bucket=bucket_name, Key=file_key)

print(f"File {file_key} uploaded to {bucket_name}")

In [ ]:
#Connect to RDS and upload .csv data
import urllib
import boto3
import os
import csv
import sys
import pymysql
import logging

s3 = boto3.client('s3')
user_name = os.environ['USER_NAME']
password = os.environ['PASSWORD']
rds_proxy_host = os.environ['RDS_PROXY_HOST']
db_name = os.environ['DB_NAME']

logger = logging.getLogger()
logger.setLevel(logging.INFO)

try:
    conn = pymysql.connect(host=rds_proxy_host, user=user_name, passwd=password, db=db_name, connect_timeout=5)
except pymysql.MySQLError as e:
    logger.error("ERROR: Unexpected error: Could not connect to MySQL instance.")
    logger.error(e)
    sys.exit(1)

logger.info("SUCCESS: Connection to RDS for MySQL instance succeeded")

def lambda_handler(event, context):
    bucket = event['Records'][0]['s3']['bucket']['name']
    key = urllib.parse.unquote_plus(event['Records'][0]['s3']['object']['key'], encoding='utf=8')
    local_file_path = '/tmp/' + os.path.basename(key)
    s3.download_file(bucket, key, local_file_path)
    logger.info("SUCCESS: File retrieved from S3") 
    
    data = []
    with open(local_file_path, 'r') as csvfile:
        csv_reader = csv.reader(csvfile)
        for row in csv_reader:
            data.append(row[:4] + [row[4].replace("Level ", '')] + row[5:])
        data = [tuple(row) for row in data]
    logger.info("SUCCESS: File read")    
    data = data[1:]
    
    sql_string = 'insert into respiratory_infections (week_ending, week, season, state, activity_level, activity_level_label) values (%s, %s, %s, %s, %s, %s)'

    with conn.cursor() as cur:
        cur.execute("create table if not exists respiratory_infections (entry_id int AUTO_INCREMENT PRIMARY KEY, week_ending date, week int, season varchar(255), state varchar(255), activity_level int, activity_level_label varchar(255))")
        cur.executemany(sql_string, data)
    conn.commit()    
    logger.info("SUCCESS: Data inserted")

In [ ]:
#Pull data from RDS and convert to dataframe
import pymysql
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

user_name = os.environ['USER_NAME']
password = os.environ['PASSWORD']
rds_proxy_host = os.environ['RDS_PROXY_HOST']
db_name = os.environ['DB_NAME']

try:
    conn = pymysql.connect(host=rds_proxy_host, user=user_name, passwd=password, db=db_name, connect_timeout=5)
except pymysql.MySQLError as e:
    print(e)

query = 'select * from respiratory_infections'
try:
    df = pd.read_sql_query(query, conn)
    df.set_index('entry_id', inplace=True)
except pd.DatabaseError as e:
    print(e)
finally:
    conn.close()

In [ ]:
sns.pairplot(df, hue="state")

In [ ]:
New_York_data_df = df[df.state == "New York"]
sns.pairplot(New_York_data_df, hue="state")

In [ ]:
Texas_data_df = df[df.state == "Texas"]
sns.pairplot(Texas_data_df, hue="state")

In [ ]:
sns.pairplot(df, hue="activity_level_label")

In [ ]:
sns.pairplot(df, hue="season")

In [ ]:
season2023_2024_data_df = df[df.season == "2023-2024"]
sns.pairplot(season2023_2024_data_df, hue="season")

In [ ]:
# Count occurrences of each state
state_counts = df['state'].value_counts().sort_values(ascending=False).head(10)

# Plot data
plt.figure(figsize=(10, 6))
state_counts.plot(kind='bar', color='skyblue')
plt.title('Top Ten States by Number of Respiratory Outbreaks')
plt.xlabel('State')
plt.ylabel('Number of Respiratory Outbreaks')
plt.xticks(rotation=45, ha='right')
plt.ylim(65, 70)  # Set y-axis limits
plt.tight_layout()

# Show plot
plt.show()

In [ ]:
# Count occurrences of each activity_level_label for each state
state_activity_counts = df.groupby(['state', 'activity_level_label']).size().unstack(fill_value=0)

# Define order of activity level labels
label_order = ['Insufficient Data', 'Minimal', 'Low', 'Moderate', 'High', 'Very High']

# Filter top ten states
top_ten_states = df['state'].value_counts().head(10).index

# Filter states and reorder columns to include only the top ten states
state_activity_counts = state_activity_counts.loc[top_ten_states]

# Reorder columns based on the specified label order
state_activity_counts = state_activity_counts[label_order]

# Plot data
plt.figure(figsize=(12, 8))
state_activity_counts.plot(kind='bar', stacked=True)
plt.title('Outbreak Severity for Top Ten States')
plt.xlabel('State')
plt.ylabel('Count')
plt.xticks(rotation=45, ha='right')
plt.legend(title='Activity Level', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()

# Show plot
plt.show()

In [ ]:
# Filter DataFrame to include only 'Very High' and 'High' activity levels
high_activity_df = df[df['activity_level_label'].isin(['Very High', 'High'])]

# Count occurrences of each state
state_counts = high_activity_df['state'].value_counts()

# Plot data
plt.figure(figsize=(10, 6))
state_counts.plot(kind='bar', color='skyblue')
plt.title('Top States with "Very High" and "High" Activity Levels')
plt.xlabel('State')
plt.ylabel('Count')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()

# Show plot
plt.show()

In [ ]:
# Filter DataFrame to include only 'Very High' and 'High' activity levels
high_activity_df = df[df['activity_level_label'].isin(['Very High', 'High'])]

# Count occurrences of each state
state_counts = high_activity_df['state'].value_counts().head(10)

# Plot data
plt.figure(figsize=(10, 6))
state_counts.plot(kind='bar', color='skyblue')
plt.title('Top Ten States with "Very High" and "High" Activity Levels')
plt.xlabel('State')
plt.ylabel('Count')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()

# Show plot
plt.show()

In [ ]:
# Filter DataFrame to include only 'Very High' and 'High' activity levels
high_activity_df = df[df['activity_level_label'].isin(['Very High', 'High'])]

# Count total number of entries with 'Very High' and 'High' activity levels by week
week_counts = high_activity_df.groupby('week').size()

# Plot data
plt.figure(figsize=(10, 6))
week_counts.plot(kind='bar', color='skyblue')
plt.title('Total Number of "Very High" and "High" Cases by Week')
plt.xlabel('Week')
plt.ylabel('Count')
plt.xticks(rotation=0)
plt.tight_layout()

# Show plot
plt.show()